In [1]:
import os
f = open("README.md", "r")
print(f.read())

# gcp-demo
Edited by Anwar



In [ ]:
import os

# folder path
dir_path = r'\\'

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
print(res)